## Problem Statement
The Vehicle Routing Problem (VRP) may be thought of as a generalization of the Travelling Salesman Problem (TSP) with $M$ salesmen instead of one. Specifically, we have depot with a certain amount of goods that need to be delivered to a number of clients located at different places. The depot has access to $M$ delivery vehicles to make these deliveries. Let $N$ be the number of clients. Consider a fully connected graph of $N + 1$ nodes with the depot being the node $0$ and the variouis clients being nodes $\{1, 2, \dots N\}$. Every edge $(i, j)$ of this fully connected graph represents the cost of moving from node $i$ to node $j$. This cost could be the time taken, the distance covered or anything at all that needs to be minimized. In our case, we have chosen it to be distance. The objective is to simply find the optimal route through this graph, for each vehicle, so that all deliveries are made with minimum cost. We use the following notation for the rest of this document:

 - *$N$: number of clients, excluding the depot*
 - *$M$: number of vehicles*
 - *$C_{ij}$: cost of moving from node $i$ to node $j$*
 - *$x_{ij}, x_{ijk}$: binary decision variables*

**Objective**: Find the optimal route for all $M$ vehicles covering all $N$ routes so that the total cost of this route is minimized.

N.B.: We have assumed here that every node is connected to every other node in the network. As we move forward we will hopefully work with a more general network.

## Full Qubo Solver (FQS)

***Representation***

$x_{i,j,k} = 1$ if $i^{th}$ vehicle is in $j^{th}$ node at the $k^{th}$ timestep.

$$i \in \{1, 2, \dots M\}; \; j \in \{0, 1, \dots N\}; \; k \in \{1, 2, \dots N\}$$

***Objective Function*** **

$$C = \sum_{m=1}^M \sum_{n=1}^N C_{0,n} x_{m,n,1} + \sum_{m=1}^M \sum_{n=1}^N C_{n,0} x_{m,n,N} + \sum_{m=1}^M \sum_{k=1}^{N-1} \sum_{i=0}^{N} \sum_{j=0}^{N} C_{i,j} x_{m,i,k} x_{m,j,k+1}$$

Here, the first term represents the cost of moving from the depot to the first visited client for each vehicle, the second term represents the cost of moving from the last visited client to the depot for each vehicle, and the third term represents the costs at the intermediate nodes.

***Constraints***

 - Every node (other than the depot) is served by exactly one vehicle at exactly one time instant. **

$$\sum_{i=1}^M \sum_{k=1}^{N(T_{max}?)} x_{i,j,k} = 1 \;\;\forall j$$

 - A vehicle can be at only one place at any given timestep.

$$\sum_{j=0}^N x_{i,j,k} = 1 \;\;\forall i, k$$

***Qubit Complexity*** **

 - No. of Logical Variables: $MN(N+1)$
 - Best Case: $N(N+1)$
 - Worst Case: $N^2(N+1)$

![](images/fqs_complexity.png)

## Average Partitioning Solver (APS)

***Representation***

$x_{i,j,k} = 1$ if $i^{th}$ vehicle is in $j^{th}$ node at the $k^{th}$ timestep.

$$i \in \{1, 2, \dots M\}; \; j \in \{0, 1, \dots N\}; \; k \in \{1, 2, \dots T\}$$

The maximum timestep $T$ is evaluated as follows

$$T = \min\{N, \frac{N}{M} + L\}$$

Here, $L$ is a solver parameter called *Limit Radius* and naturally, we round up the above $T$ to the next integer.

***Objective Function*** **

$$C = \sum_{m=1}^M \sum_{n=1}^N C_{0,n} x_{m,n,1} + \sum_{m=1}^M \sum_{n=1}^N C_{n,0} x_{m,n,T} + \sum_{m=1}^M \sum_{n=1}^{T-1} \sum_{i=0}^{N} \sum_{j=0}^{N} C_{i,j} x_{m,i,n} x_{m,j,n+1}$$

Here, the first term represents the cost of moving from the depot to the first visited client for each vehicle, the second term represents the cost of moving from the last visited client to the depot for each vehicle, and the third term represents the costs at the intermediate nodes.

***Constraints***

 - Every node (other than the depot) is served by exactly one vehicle at exactly one time instant.

$$\sum_{i=1}^M \sum_{j=1}^T x_{i,k,j} = 1 \;\;\forall k$$

 - A vehicle can be at only one place at any given timestep.

$$\sum_{k=0}^N x_{i,k,j} = 1 \;\;\forall i, j$$

***Qubit Complexity*** **

 - No. of Logical Variables: $MT(N+1)$
 - Best Case: $N(N+1)$
 - Worst Case: $N^2(N+1)$

![](images/aps_complexity.png)

## Route Activation Solver (RAS)

***Representation***

$x_{i,j} = 1$ if the route from the $i^{th}$ node to the $j^{th}$ node is being traversed by any vehicle at any time. Where $i \in \{0, 1, \dots N\}, \ j \in \{0, 1, \dots N\}, \ i \neq j$

***Objective Function***

$$C = \sum_{i=0}^N \sum_{j=0}^N C_{i,j} x_{i,j}$$

***Constraints***

 - Each node other than the depot has exactly $1$ outgoing active edge and exactly $1$ incoming active edge.

$$\sum_{j\neq i,j=0}^N x_{i,j} = 1, \sum_{j\neq i,j=0}^N x_{j,i} = 1, \ \forall i$$

 - The depot has exactly $M$ outgoing and $M$ incoming connections.

$$\sum_{i=1}^N x_{0,i} = M, \ \ \sum_{i=1}^N x_{i,0} = M$$

 - **The routes must be connected and that the demand on each route must not exceed the vehicle capacity.**
$$\sum_{i \in S}\sum_{j \in N-S} x_{ij} \ge r(S) \forall S ⊂ {1,2,...,n}; \ (r(S)=1 \ in \ our \ case)$$
$$\sum_{i \in N-S}\sum_{j \in S} x_{ij} \ge r(S) \forall S ⊂ {1,2,...,n}; \ (r(S)=1 \ in \ our \ case)$$

∑i∈S∑j∈N−Sxij≥1(ϕ≠S⊂{1,…,n},|S|≥2
∑i∈N−S∑j∈Sxij≥1(ϕ≠S⊂{1,…,n},|S|≥2

***Qubit Complexity*** **

 - No. of Logical Variables: $N(N+1)$
 - Best Case: $N(N+1)$
 - Worst Case: $N(N+1)$

![](images/ras_complexity.png)

[**]: Needs Verification